In [12]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [13]:
N_centr = 4
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_56th_new_19668_702runs.root"] 
#m_ee_Run14AuAu_50th_new_19640_1072runs
#m_ee_Run14AuAu_48th_new_19631_387runs m_ee_Run14AuAu_49th_new_19636_1072runs
#m_ee_Run14AuAu_46th_new_19619_999runs test_BG_v94_0 m_ee_Run14AuAu_47th_new_19623_1078runs 
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['ccbar_50M_v2.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_50M_v8.root']
conv_file_names=['conv_100M.root']
sim_file_names = [primary_file_names[0]]
legend_names = ["pos","neg","sim_pos","sim_neg"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3",\
                        "myvtx_hist_0","myvtx_hist_1","myvtx_hist_2",'myvtx_hist_3']

In [14]:
N_hists = len(hist_select_3D_names)

In [15]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_56th_new_19668_702runs.root
../sim/output/Newembed/sngl/ccbar_50M_v2.root


In [16]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1)
    neg_proj = hists_read[iFile][type_loc+4].ProjectionX(hists_read[iFile][type_loc+4].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc+4].ProjectionX(hists_sim[iFile][type_loc+4].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [17]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="x, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(0.27),pos_proj.FindBin(0.4))/pos_proj.Integral())

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9928839438744739
0.9931667073977848
0.9929390628618289
0.989781501022673


In [18]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[iFile][type_loc].ProjectionY(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="y, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(-0.02),pos_proj.FindBin(0.09))/pos_proj.Integral())

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9999505007433994
0.9998817649866303
0.9997940463002749
0.998689253070178


In [19]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[iFile][type_loc].ProjectionZ(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="y, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")